In [1]:
import os
os.chdir('..')

In [2]:
from kernel.type import TFun, BoolType, NatType
from kernel.term import Var, Implies, Forall, BoolVars
from kernel.proofterm import ProofTerm, TacticException
from kernel.thm import Thm
from logic import basic
from logic.tactic import Tactic
from syntax.settings import settings

basic.load_theory('nat')
settings.unicode = True

## Tactics

So far our proofs are constructed mostly in the bottom-up fashion. That is, the proof term is built up by first building its components, then composing these components together. Often, a different approach, constructing a proof top-down, is more appropriate. In this approach, a goal is reduced to one or more simpler subgoals, which are further reduced, until there is no subgoal remaining. The system of *tactics* permits this kind of reasoning.

A tactic is a function that takes a theorem as input, and returns a proof term proving that theorem. The returned proof term may contain gaps (constructed by the rule `sorry`), indicating remaining goals to be proved. Hence, a tactic can also be viewed as transforming a goal to a list of subgoals. Like conversions, we can compose tactics together to form more complex tactics using *tacticals*.

Tactics is represented in holpy as the class `Tactic`. New tactics are implemented by inheriting from this class. A tactic can take additional arguments through its `__init__` function. The `get_proof_term` function takes a goal (of type `Thm`) as argument, and should return a proof term whose conclusion is the goal. The returned proof term may contain gaps.

Let's begin with some simple examples of tactics. The `intro_imp_tac` tactic takes an implication goal, and moves one assumption to the hypothesis. It applies the following proof rule in the reverse direction:

$$ \frac{\Gamma, A \vdash C}{\Gamma \vdash A\to C} $$

In [3]:
class intro_imp_tac(Tactic):
    def get_proof_term(self, goal):
        if not goal.prop.is_implies():
            raise TacticException('intro_imp: goal is not implies.')

        A, C = goal.prop.args
        new_goal = ProofTerm.sorry(Thm(list(goal.hyps) + [A], C))
        return new_goal.implies_intr(A)

We can test the tactic directly on a theorem (note the printing function for a proof term also prints the remaining goals to be proved).

In [4]:
x = Var('x', NatType)
P = Var('P', TFun(NatType, BoolType))
Q = Var('Q', TFun(NatType, BoolType))
A = Var('A', BoolType)

th = Thm([A], Implies(P(x), Q(x)))
print('goal:', th)

pt = intro_imp_tac().get_proof_term(th)
print('pt:', pt)

print('\nProof:')
print(pt.export())

goal: A ⊢ P x ⟶ Q x
pt: ProofTerm(A ⊢ P x ⟶ Q x)
Gaps: A, P x ⊢ Q x

Proof:
0: A, P x ⊢ Q x by sorry
1: A ⊢ P x ⟶ Q x by implies_intr P x from 0


The `intro_forall_tac` takes an forall goal, and introduces a new variable. If a variable name is provided it is used. Otherwise the name provided by the abstraction is used. It applies the following proof rule in the reverse direction:

$$ \frac{\Gamma \vdash P(a)}{\Gamma \vdash \forall a.\,P(a)}\hbox{ ($a$ does not occur in $\Gamma$)} $$

In [5]:
class intro_forall_tac(Tactic):
    def __init__(self, var_name=None):
        self.var_name = var_name
        
    def get_proof_term(self, goal):
        if not goal.prop.is_forall():
            raise TacticException('intro_forall: goal is not forall')

        v, body = goal.prop.arg.dest_abs(self.var_name)
        new_goal = ProofTerm.sorry(Thm(goal.hyps, body))
        return new_goal.forall_intr(v)

In [6]:
th = Thm([A], Forall(x, Implies(P(x), Q(x))))
print('goal:', th)

pt = intro_forall_tac().get_proof_term(th)
print('pt:', pt)

print('\nProof:')
print(pt.export())

goal: A ⊢ ∀x. P x ⟶ Q x
pt: ProofTerm(A ⊢ ∀x. P x ⟶ Q x)
Gaps: A ⊢ P x ⟶ Q x

Proof:
0: A ⊢ P x ⟶ Q x by sorry
1: A ⊢ ∀x. P x ⟶ Q x by forall_intr x from 0


A tactic can also be applied on a proof term, through the proof term's `tac` method. The method applies the tactic to the *first* subgoal of the proof term.

In [7]:
pt = ProofTerm.sorry(Thm([A], Implies(P(x), Q(x))))
print('pt:', pt)

pt2 = pt.tac(intro_imp_tac())
print('pt2:', pt2)

pt: ProofTerm(A ⊢ P x ⟶ Q x)
Gaps: A ⊢ P x ⟶ Q x
pt2: ProofTerm(A ⊢ P x ⟶ Q x)
Gaps: A, P x ⊢ Q x


Several tactics can be applied in sequence using the `tacs` method:

In [8]:
pt = ProofTerm.sorry(Thm([A], Forall(x, Implies(P(x), Q(x)))))
print('pt:', pt)

pt2 = pt.tacs(intro_forall_tac('a'), intro_imp_tac())
print('pt2:', pt2)

pt: ProofTerm(A ⊢ ∀x. P x ⟶ Q x)
Gaps: A ⊢ ∀x. P x ⟶ Q x
pt2: ProofTerm(A ⊢ ∀x. P x ⟶ Q x)
Gaps: A, P x ⊢ Q x


## Tacticals

A tactical can be used to compose together tactics to form more complex tactics. This works in a way similar to conversion combinators. In holpy, a tactical is also implemented by inheriting from class `Tactic`, taking the component tactics as arguments to the `__init__` function. We first demonstrate this with the `then_tac` tactical:

In [9]:
class then_tac(Tactic):
    def __init__(self, tac1, tac2):
        self.tac1 = tac1
        self.tac2 = tac2
        
    def get_proof_term(self, goal):
        return ProofTerm.sorry(goal).tacs(self.tac1, self.tac2)

Use `tacs` to apply a sequence of tactics is the same as using `tac` to apply the `then_tac` of these tactics. For example:

In [10]:
pt = ProofTerm.sorry(Thm([A], Forall(x, Implies(P(x), Q(x)))))
print('pt:', pt)

pt2 = pt.tac(then_tac(intro_forall_tac('a'), intro_imp_tac()))
print('pt2:', pt2)

pt: ProofTerm(A ⊢ ∀x. P x ⟶ Q x)
Gaps: A ⊢ ∀x. P x ⟶ Q x
pt2: ProofTerm(A ⊢ ∀x. P x ⟶ Q x)
Gaps: A, P x ⊢ Q x


The tactical `else_tac` tries to apply the first tactic, and if fails, the second tactics:

In [11]:
class else_tac(Tactic):
    def __init__(self, tac1, tac2):
        self.tac1 = tac1
        self.tac2 = tac2
        
    def get_proof_term(self, goal):
        try:
            return self.tac1.get_proof_term(goal)
        except TacticException:
            return self.tac2.get_proof_term(goal)

For example, we can form the tactic `intro_one`, which introduces either an implication or a forall quantification:

In [12]:
intro_one_tac = else_tac(intro_imp_tac(), intro_forall_tac())

print(intro_one_tac.get_proof_term(Thm([A], Implies(P(x), Q(x)))))
print(intro_one_tac.get_proof_term(Thm([A], Forall(x, Implies(P(x), Q(x))))))

ProofTerm(A ⊢ P x ⟶ Q x)
Gaps: A, P x ⊢ Q x
ProofTerm(A ⊢ ∀x. P x ⟶ Q x)
Gaps: A ⊢ P x ⟶ Q x


The tactical `repeat_tac` repeatedly applies some tactic, until `TacticException` is raised:

In [13]:
class repeat_tac(Tactic):
    def __init__(self, tac):
        self.tac = tac
        
    def get_proof_term(self, goal):
        pt = ProofTerm.sorry(goal)
        while True:
            try:
                pt = pt.tac(self.tac)
            except TacticException:
                break
        return pt

The tactic `intros_tac` repeatedly introduces implication and forall quantification:

In [14]:
intros_tac = repeat_tac(else_tac(intro_imp_tac(), intro_forall_tac()))

print(intros_tac.get_proof_term(Thm([A], Forall(x, Implies(P(x), Q(x))))))

ProofTerm(A ⊢ ∀x. P x ⟶ Q x)
Gaps: A, P x ⊢ Q x


Implication and forall quantification can also alternate:

In [15]:
x = Var('x', NatType)
y = Var('y', NatType)
P = Var('P', TFun(NatType, BoolType))
Q = Var('Q', TFun(NatType, BoolType))
R = Var('R', TFun(NatType, NatType, BoolType))

goal = Forall(x, Implies(P(x), Forall(y, Implies(Q(y), R(x, y)))))
print('goal:', goal)
print(intros_tac.get_proof_term(Thm([], goal)))

goal: ∀x. P x ⟶ (∀y. Q y ⟶ R x y)
ProofTerm(⊢ ∀x. P x ⟶ (∀y. Q y ⟶ R x y))
Gaps: P x, Q y ⊢ R x y
